In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [6]:
cmd_pt_df = pd.read_csv('ind_passthrough_estimates.csv')

In [11]:
cmd_pt_df

,dms_orig,dms_dest,dms_mode,sctg2,trade_type,dist_band,tons_2018,tons_2019,tons_2020,tons_2021,...,dms_dest_desc,dms_dest_lat,dms_dest_lon,Commodity_Type,Distance_Band,counties_crossed,crosses_FW_counties,crosses_IND_counties,crosses_Gary_counties,any_IN_counties
0,172,392,2,2,1,4,0.088608,0.072417,0.082935,0.085977,...,Cleveland OH,41.499657,-81.693677,Cereal grains,500 - 749,"['Hamilton', 'Randolph', 'Vigo', 'Delaware', '...",False,True,False,True
1,179,131,2,2,1,4,259.605309,212.168824,242.984359,251.896342,...,Atlanta GA,33.748992,-84.390264,Cereal grains,500 - 749,"['Posey', 'Gibson', 'Vanderburgh']",False,False,False,True
2,179,139,2,2,1,5,567.403683,463.724615,531.076275,550.554658,...,Rest of GA,32.329381,-83.113737,Cereal grains,750 - 999,"['Posey', 'Gibson', 'Vanderburgh']",False,False,False,True
3,179,249,2,2,1,5,0.509261,0.416206,0.476656,0.494139,...,Rest of MD,39.516240,-76.938207,Cereal grains,750 - 999,"['Hamilton', 'Montgomery', 'Vermillion', 'Henr...",False,True,False,True
4,179,361,2,2,1,5,0.745495,0.609274,0.697766,0.723358,...,Albany NY CFS Area,42.651167,-73.754968,Cereal grains,750 - 999,"['Huntington', 'Wabash', 'Wells', 'Warren', 'T...",True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62902,279,451,2,43,3,6,0.000000,0.000000,0.000701,0.002025,...,Charleston SC,32.788436,-79.939931,Mixed freight,"1,000 - 1,499","['Orange', 'Vermillion', 'Greene', 'Owen', 'Vi...",False,False,False,True
62903,279,512,2,43,3,6,0.000000,0.000000,0.000000,0.005049,...,Virginia Beach-Norfolk VA-NC (VA Part),36.849658,-75.976075,Mixed freight,"1,000 - 1,499","['Allen', 'Elkhart', 'LaGrange', 'Noble', 'DeK...",True,False,False,True
62904,292,512,2,43,3,6,0.000000,0.001006,0.001119,0.000000,...,Virginia Beach-Norfolk VA-NC (VA Part),36.849658,-75.976075,Mixed freight,"1,000 - 1,499","['Perry', 'Dubois', 'Gibson', 'Harrison', 'Cra...",False,False,False,True
62905,392,61,2,43,3,8,0.000000,0.000000,0.000000,0.000508,...,Los Angeles CA,34.053691,-118.242766,Mixed freight,"Over 2,000","['Huntington', 'Wabash', 'Wells', 'Warren', 'T...",True,False,False,True


In [15]:
cmd_pt_FW_df = cmd_pt_df[cmd_pt_df['crosses_FW_counties'] == True][['sctg2', 'tons_2022']]
cmd_pt_FW_df = cmd_pt_FW_df.groupby(['sctg2']).\
                agg(
                    {'tons_2022': 'sum'})\
                .reset_index()
cmd_pt_FW_df['dms_loc'] = 183

cmd_pt_IN_df = cmd_pt_df[cmd_pt_df['crosses_IND_counties'] == True][['sctg2', 'tons_2022']]
cmd_pt_IN_df = cmd_pt_IN_df.groupby(['sctg2']).\
                agg(
                    {'tons_2022': 'sum'})\
                .reset_index()
cmd_pt_IN_df['dms_loc'] = 182

cmd_pt_Gary_df = cmd_pt_df[cmd_pt_df['crosses_Gary_counties'] == True][['sctg2', 'tons_2022']]
cmd_pt_Gary_df = cmd_pt_Gary_df.groupby(['sctg2']).\
                agg(
                    {'tons_2022': 'sum'})\
                .reset_index()
cmd_pt_Gary_df['dms_loc'] = 181

cmd_pt_ROI_df = cmd_pt_df[['sctg2', 'tons_2022']]
cmd_pt_ROI_df = cmd_pt_ROI_df.groupby(['sctg2']).\
                agg(
                    {'tons_2022': 'sum'})\
                .reset_index()
cmd_pt_ROI_df['dms_loc'] = 189
#IN_Counties['dms_code'] = np.where(IN_Counties['NAME'].str.contains('|'.join(FW_Cnt), case = False), 183,
#                                            np.where(IN_Counties['NAME'].str.contains('|'.join(Gary_Cnt), case = False), 181,
#                                                     np.where(IN_Counties['NAME'].str.contains('|'.join(Ind_Cnt), case = False), 182,
#                                                              189)))

In [17]:
cmd_dms_loc_pt_df = pd.concat([cmd_pt_ROI_df, cmd_pt_FW_df, cmd_pt_IN_df, cmd_pt_Gary_df], ignore_index=True)


cmd_dms_ref_df = cmd_dms_loc_pt_df.groupby('dms_loc').agg({'tons_2022': 'sum'})\
                .reset_index()\
                .rename(columns = {'tons_2022': 'total_tons_2022'})


cmd_dms_pt_df_final = pd.merge(cmd_dms_loc_pt_df, cmd_dms_ref_df, on = 'dms_loc', how = 'left')

In [18]:
cmd_dms_pt_df_final

,sctg2,tons_2022,dms_loc,total_tons_2022
0,1,0.002117,189,150841.540705
1,2,20721.114806,189,150841.540705
2,3,6298.601740,189,150841.540705
3,4,2491.920671,189,150841.540705
4,5,296.959666,189,150841.540705
...,...,...,...,...
160,38,39.827269,181,31997.163193
161,39,28.193830,181,31997.163193
162,40,81.976174,181,31997.163193
163,41,1098.257031,181,31997.163193
